In [499]:
import requests
import pandas as pd
from datetime import date, timedelta
import json

In [500]:
start_date = date.today() - timedelta(7)
end_date = date.today() - timedelta(1)

url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={start_date}&endtime={end_date}"

print(url)

https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2025-02-07&endtime=2025-02-13


In [501]:
# USGS Earthquake API URL
url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={start_date}&endtime={end_date}"

# Extracting from API
response = requests.get(url)

# Checking status code
if response.status_code == 200:
    data = response.json()
    print("API fetch successfull")
else:
    print("Error fetching json data")


API fetch successfull


In [502]:
data_raw = pd.json_normalize(data['features'])
data_raw

,type,id,properties.mag,properties.place,properties.time,properties.updated,properties.tz,properties.url,properties.detail,properties.felt,...,properties.types,properties.nst,properties.dmin,properties.rms,properties.gap,properties.magType,properties.type,properties.title,geometry.type,geometry.coordinates
0,Feature,tx2025daop,2.70,"34 km SE of Malaga, New Mexico",1739404758069,1739406380640,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,",origin,phase-data,",33.0,0.00000,0.10,28.0,ml,earthquake,"M 2.7 - 34 km SE of Malaga, New Mexico",Point,"[-103.805, 32.007, 7.0868]"
1,Feature,ak0251zi5e0s,1.90,"30 km NW of Willow, Alaska",1739404710361,1739404833785,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,",origin,phase-data,",NaN,NaN,0.48,NaN,ml,earthquake,"M 1.9 - 30 km NW of Willow, Alaska",Point,"[-150.4411, 61.9425, 29.4]"
2,Feature,uw62067147,1.35,"10 km ENE of Abbotsford, Canada",1739403975790,1739562672990,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,",origin,phase-data,",7.0,0.08403,0.21,230.0,ml,explosion,"M 1.4 Explosion - 10 km ENE of Abbotsford, Canada",Point,"[-122.11666666666666, 49.07966666666667, -0.66]"
3,Feature,ak0251zi2nmi,0.30,"59 km W of Tyonek, Alaska",1739403954105,1739407096160,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,",origin,phase-data,",NaN,NaN,0.54,NaN,ml,earthquake,"M 0.3 - 59 km W of Tyonek, Alaska",Point,"[-152.2285, 61.1445, 15.8]"
4,Feature,nc75132492,0.73,"22 km NE of San Ardo, CA",1739403815660,1739563479165,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,",nearby-cities,origin,phase-data,",11.0,0.06236,0.10,145.0,md,earthquake,"M 0.7 - 22 km NE of San Ardo, CA",Point,"[-120.744666666667, 36.1691666666667, 5.39]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1861,Feature,uw62066027,0.15,"22 km ENE of Ashford, Washington",1738887407640,1739392500710,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,",origin,phase-data,",18.0,0.01898,0.08,70.0,ml,earthquake,"M 0.2 - 22 km ENE of Ashford, Washington",Point,"[-121.76533333333333, 46.85183333333333, 1.75]"
1862,Feature,ak0251quhu1f,1.40,"94 km N of Karluk, Alaska",1738887154756,1738957956620,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,",origin,phase-data,",NaN,NaN,0.52,NaN,ml,earthquake,"M 1.4 - 94 km N of Karluk, Alaska",Point,"[-154.6902, 58.4053, 5.6]"
1863,Feature,ci40181162,1.85,"12 km SE of Tehachapi, CA",1738886902020,1738951984410,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,",focal-mechanism,nearby-cities,origin,phase-da...",36.0,0.07331,0.15,38.0,ml,earthquake,"M 1.9 - 12 km SE of Tehachapi, CA",Point,"[-118.3488333, 35.0638333, 4.69]"
1864,Feature,ak0251qugrut,1.80,"59 km E of Port Alsworth, Alaska",1738886859930,1738968119040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,",origin,phase-data,",NaN,NaN,0.28,NaN,ml,earthquake,"M 1.8 - 59 km E of Port Alsworth, Alaska",Point,"[-153.241, 60.1802, 150.8]"


## Transformation

In [504]:
# 1. Drop Unwanted Columns
# 2. Convert properties.time and updated into date time
# 3. Split Coordinates
# 4. Concat coordinates and cleaned df
# 5.Strip Propoerties.types
# 6. Drop Duplicates
# 7. Reorder columns

In [505]:
#1 Drop Unwanted Columns

data_cleaned = data_raw.drop(columns = ['properties.detail','properties.url','properties.tz', 
                                    'properties.felt','properties.nst','properties.gap',
                                    'properties.dmin','geometry.type','properties.cdi',
                                    'properties.mmi','properties.alert','properties.status',
                                    'properties.net','properties.code','properties.ids',
                                     'properties.sources','properties.tsunami','properties.types',
                                    'properties.rms','properties.type'
                                    ])

In [506]:
data_cleaned.head(5)

,type,id,properties.mag,properties.place,properties.time,properties.updated,properties.sig,properties.magType,properties.title,geometry.coordinates
0,Feature,tx2025daop,2.70,"34 km SE of Malaga, New Mexico",1739404758069,1739406380640,112,ml,"M 2.7 - 34 km SE of Malaga, New Mexico","[-103.805, 32.007, 7.0868]"
1,Feature,ak0251zi5e0s,1.90,"30 km NW of Willow, Alaska",1739404710361,1739404833785,56,ml,"M 1.9 - 30 km NW of Willow, Alaska","[-150.4411, 61.9425, 29.4]"
2,Feature,uw62067147,1.35,"10 km ENE of Abbotsford, Canada",1739403975790,1739562672990,28,ml,"M 1.4 Explosion - 10 km ENE of Abbotsford, Canada","[-122.11666666666666, 49.07966666666667, -0.66]"
3,Feature,ak0251zi2nmi,0.30,"59 km W of Tyonek, Alaska",1739403954105,1739407096160,1,ml,"M 0.3 - 59 km W of Tyonek, Alaska","[-152.2285, 61.1445, 15.8]"
4,Feature,nc75132492,0.73,"22 km NE of San Ardo, CA",1739403815660,1739563479165,8,md,"M 0.7 - 22 km NE of San Ardo, CA","[-120.744666666667, 36.1691666666667, 5.39]"


In [507]:
# Dividing y 1000 to convert into seconds
data_cleaned['properties.time'] = data_cleaned['properties.time'] / 1000
data_cleaned['properties.updated'] = data_cleaned['properties.updated'] / 1000

# Converting the columns into datetime
data_cleaned['properties.time'] = pd.to_datetime(data_cleaned['properties.time'],unit = 's')
data_cleaned['properties.updated'] = pd.to_datetime(data_cleaned['properties.updated'],unit = 's')

In [508]:
data_cleaned.head(5)

,type,id,properties.mag,properties.place,properties.time,properties.updated,properties.sig,properties.magType,properties.title,geometry.coordinates
0,Feature,tx2025daop,2.70,"34 km SE of Malaga, New Mexico",2025-02-12 23:59:18.069000006,2025-02-13 00:26:20.640000105,112,ml,"M 2.7 - 34 km SE of Malaga, New Mexico","[-103.805, 32.007, 7.0868]"
1,Feature,ak0251zi5e0s,1.90,"30 km NW of Willow, Alaska",2025-02-12 23:58:30.361000061,2025-02-13 00:00:33.785000086,56,ml,"M 1.9 - 30 km NW of Willow, Alaska","[-150.4411, 61.9425, 29.4]"
2,Feature,uw62067147,1.35,"10 km ENE of Abbotsford, Canada",2025-02-12 23:46:15.789999962,2025-02-14 19:51:12.990000010,28,ml,"M 1.4 Explosion - 10 km ENE of Abbotsford, Canada","[-122.11666666666666, 49.07966666666667, -0.66]"
3,Feature,ak0251zi2nmi,0.30,"59 km W of Tyonek, Alaska",2025-02-12 23:45:54.105000019,2025-02-13 00:38:16.160000086,1,ml,"M 0.3 - 59 km W of Tyonek, Alaska","[-152.2285, 61.1445, 15.8]"
4,Feature,nc75132492,0.73,"22 km NE of San Ardo, CA",2025-02-12 23:43:35.660000086,2025-02-14 20:04:39.164999962,8,md,"M 0.7 - 22 km NE of San Ardo, CA","[-120.744666666667, 36.1691666666667, 5.39]"


In [509]:
data_cleaned[['Longitude', 'Latitude', 'Elevation']] = pd.DataFrame(data_cleaned['geometry.coordinates'].to_list(), index=data_cleaned.index)
data_cleaned = data_cleaned.drop(columns =['geometry.coordinates'])

In [510]:
data_cleaned.head(5)

,type,id,properties.mag,properties.place,properties.time,properties.updated,properties.sig,properties.magType,properties.title,Longitude,Latitude,Elevation
0,Feature,tx2025daop,2.70,"34 km SE of Malaga, New Mexico",2025-02-12 23:59:18.069000006,2025-02-13 00:26:20.640000105,112,ml,"M 2.7 - 34 km SE of Malaga, New Mexico",-103.805000,32.007000,7.0868
1,Feature,ak0251zi5e0s,1.90,"30 km NW of Willow, Alaska",2025-02-12 23:58:30.361000061,2025-02-13 00:00:33.785000086,56,ml,"M 1.9 - 30 km NW of Willow, Alaska",-150.441100,61.942500,29.4000
2,Feature,uw62067147,1.35,"10 km ENE of Abbotsford, Canada",2025-02-12 23:46:15.789999962,2025-02-14 19:51:12.990000010,28,ml,"M 1.4 Explosion - 10 km ENE of Abbotsford, Canada",-122.116667,49.079667,-0.6600
3,Feature,ak0251zi2nmi,0.30,"59 km W of Tyonek, Alaska",2025-02-12 23:45:54.105000019,2025-02-13 00:38:16.160000086,1,ml,"M 0.3 - 59 km W of Tyonek, Alaska",-152.228500,61.144500,15.8000
4,Feature,nc75132492,0.73,"22 km NE of San Ardo, CA",2025-02-12 23:43:35.660000086,2025-02-14 20:04:39.164999962,8,md,"M 0.7 - 22 km NE of San Ardo, CA",-120.744667,36.169167,5.3900


In [511]:
data_cleaned = data_cleaned.rename(columns={'properties.mag':'mag',
                             'properties.place':'place',
                             'properties.time':'time',
                            'properties.updated':'updated',
                             'properties.sig':'sig',
                             'properties.magType':'magType',
                            'properties.title':'title'})
data_cleaned.head(5)

,type,id,mag,place,time,updated,sig,magType,title,Longitude,Latitude,Elevation
0,Feature,tx2025daop,2.70,"34 km SE of Malaga, New Mexico",2025-02-12 23:59:18.069000006,2025-02-13 00:26:20.640000105,112,ml,"M 2.7 - 34 km SE of Malaga, New Mexico",-103.805000,32.007000,7.0868
1,Feature,ak0251zi5e0s,1.90,"30 km NW of Willow, Alaska",2025-02-12 23:58:30.361000061,2025-02-13 00:00:33.785000086,56,ml,"M 1.9 - 30 km NW of Willow, Alaska",-150.441100,61.942500,29.4000
2,Feature,uw62067147,1.35,"10 km ENE of Abbotsford, Canada",2025-02-12 23:46:15.789999962,2025-02-14 19:51:12.990000010,28,ml,"M 1.4 Explosion - 10 km ENE of Abbotsford, Canada",-122.116667,49.079667,-0.6600
3,Feature,ak0251zi2nmi,0.30,"59 km W of Tyonek, Alaska",2025-02-12 23:45:54.105000019,2025-02-13 00:38:16.160000086,1,ml,"M 0.3 - 59 km W of Tyonek, Alaska",-152.228500,61.144500,15.8000
4,Feature,nc75132492,0.73,"22 km NE of San Ardo, CA",2025-02-12 23:43:35.660000086,2025-02-14 20:04:39.164999962,8,md,"M 0.7 - 22 km NE of San Ardo, CA",-120.744667,36.169167,5.3900


In [512]:
#Defining Sig Class

def sig_class:
    if sig_class < 100:
        return 'low'
elif sig_class >= 100

SyntaxError: expected '(' (817240857.py, line 3)